# Basics



### Variables

In [1]:
import torch

# Single number
t1 = torch.tensor(4.)
t1

tensor(4.)

In [2]:
t1.dtype

torch.float32

In [3]:
# Vector
t2 = torch.tensor([1., 2, 3, 4])
t2

tensor([1., 2., 3., 4.])

In [4]:
# Matrix
t3 = torch.tensor([[1., 2, 3],
                   [4, 5, 6]])
t3

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [6]:
# 3-dimensional array
t4 = torch.tensor([[[11., 12, 13],
                    [14, 15, 16]],
                   
                   [[5, 6, 7],
                    [8, 9, 10]]])
t4

tensor([[[11., 12., 13.],
         [14., 15., 16.]],

        [[ 5.,  6.,  7.],
         [ 8.,  9., 10.]]])

### Shapes

In [7]:
t1.shape

torch.Size([])

In [8]:
t2.shape

torch.Size([4])

In [9]:
t3.shape

torch.Size([2, 3])

In [10]:
t4.shape

torch.Size([2, 2, 3])

### Tensors' operations

In [12]:
# Create tensors
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

In [13]:
# Arithmetic operation
y = x * w + b
y

tensor(17., grad_fn=<AddBackward0>)

In [14]:
# Compute derivatives
y.backward()

In [16]:
# Display gradients (derivatives)
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


### Interaction with Numpy

In [18]:
import numpy as np

x = np.array([[1, 2],
              [3, 4.]])
x

array([[1., 2.],
       [3., 4.]])

In [19]:
# Convert into tensor
y = torch.tensor(x)
y

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [20]:
# Convert back
z = y.numpy()
z

array([[1., 2.],
       [3., 4.]])

# Linear Regression with PyTorch

### Libraries

In [25]:
import torch
import torch.nn as nn
import numpy as np

### Data in arrays

In [26]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [74, 66, 43], 
                   [91, 87, 65], [88, 134, 59], [101, 44, 37], 
                   [68, 96, 71], [73, 66, 44], [92, 87, 64], 
                   [87, 135, 57], [103, 43, 36], [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119],[57, 69], 
                    [80, 102], [118, 132], [21, 38], 
                    [104, 118], [57, 69], [82, 100], 
                    [118, 134], [20, 38], [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Data in PyTorch built-ins

In [27]:
from torch.utils.data import TensorDataset

# Define Dataset
train_ds = TensorDataset(inputs, targets)
# Inputs
train_ds[:3][0]

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.]])

In [28]:
# Targets
train_ds[:3][1]

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.]])

DataLoader can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [29]:
from torch.utils.data import DataLoader

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [30]:
# Our first batch
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [101.,  44.,  37.]])
tensor([[118., 134.],
        [ 20.,  38.],
        [119., 133.],
        [ 22.,  37.],
        [ 21.,  38.]])


### Model

In [31]:
# Define model
model = nn.Linear(3, 2) # We have 3 input features and 2 outputs
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5304,  0.0973, -0.0329],
        [-0.4385, -0.3344, -0.5466]], requires_grad=True)
Parameter containing:
tensor([0.1240, 0.2582], requires_grad=True)


### Loss function

In [32]:
import torch.nn.functional as F

loss_fn = F.mse_loss

In [33]:
loss = loss_fn(model(inputs), targets)
loss

tensor(19619.5684, grad_fn=<MseLossBackward0>)

### Optimizer

In [34]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### Utility function to train the model

We use DataLoader defined earlier however we can use different models, losses, optimizers and number of epochs.

In [35]:
def fit(model, loss_fn, opt, num_epochs=100):

    # Several epochs
    for epoch in range(num_epochs):

        # Train with batches of data
        for xb, yb in train_dl:

            # 1. Generate predictions
            preds = model(xb)

            # 2. Calculate loss
            loss = loss_fn(preds, yb)

            # 3. Compute gradients
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset gradients back to zero
            opt.zero_grad()
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

In [36]:
fit(model, loss_fn, opt)

Epoch 10/100, Loss: 320.4903
Epoch 20/100, Loss: 538.9030
Epoch 30/100, Loss: 118.4743
Epoch 40/100, Loss: 260.2313
Epoch 50/100, Loss: 126.1844
Epoch 60/100, Loss: 53.1091
Epoch 70/100, Loss: 122.2629
Epoch 80/100, Loss: 64.9025
Epoch 90/100, Loss: 65.4798
Epoch 100/100, Loss: 22.9508
